### 0: Imports

In [2]:
import datetime
import praw
from pmaw import PushshiftAPI
import mysql.connector
from mysql.connector import Error
import re

### 1: API Infos

In [3]:
info_file = open("api info.txt", "r")
id = info_file.readline().replace("\n", "")
secret = info_file.readline().replace("\n", "")
ua = info_file.readline().replace("\n", "")

### 2: Initialisation

In [4]:
reddit = praw.Reddit(
 client_id=id,
 client_secret=secret,
 user_agent=ua
)
api_praw = PushshiftAPI(praw=reddit)

before = int(datetime.datetime(2016,3,29,0,0).timestamp())
after = int(datetime.datetime(2015,3,29,0,0).timestamp())

### 3: Fetching Data

In [10]:
posts = api_praw.search_submissions(subreddit="france", until=before, since=after)
post_list = list(posts)
print(f"Total posts scraped : {len(post_list)}")

Total posts scraped : 27059


### 4: DB functions

In [11]:
def create_db_connection(host, user, password, db):
    connection = None
    try:
        connection = mysql.connector.connect(
            host = host,
            user = user,
            passwd = password,
            database = db
        )
        print("MySQL Database connection established")
    except Error as err:
        print(f"Error: '{err}'")

    return connection

def execute_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        return False
    except Error as err:
        #print(f"Error: '{err}'")
        return True


def execute_query_get_result(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        L = cursor.fetchall()
        connection.commit()
    except Error as err:
        print(f"Error: '{err}'")
    return L

### 5: DB connection

In [6]:
connection = create_db_connection(XXX) # Connect to the Database

MySQL Database connection established


### 6: Scraping posts

In [12]:
for x in post_list:
    title = x['title'].replace("\"","")
    body = x['selftext'].replace("\"","")
    ins = f"INSERT INTO post VALUES (\"{x['id']}\", {x['over_18']}, {str(x['num_comments'] > 0)}, {str(x['selftext'] == '[deleted]')},\"{title}\", \"{body}\");"
    execute_query(connection, ins)

Error: '1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near '"Comment devenir un exclu. (vision cynique d'un SDF) /!\ WARNING WEB 1.0 /!\", "' at line 1'
Error: '1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near '[deleted]")' at line 1'


### 7: Scraping comments

In [14]:
def clean_text(text):
    return re.sub(r'[^\w\s]', '', text)
    
def explore_and_insert_comments(commentForest, postID):
    for comment in commentForest:
        replies = comment.replies
        parent = comment.parent_id
        if parent.startswith('t3_'):
            ins = f"INSERT INTO comments VALUES (\"{comment.id}\", {str(len(replies) > 0)}, \"{clean_text(comment.body)}\", \"{postID}\", NULL);"
        else:
            ins = f"INSERT INTO comments VALUES (\"{comment.id}\", {str(len(replies) > 0)}, \"{clean_text(comment.body)}\", \"{postID}\", \"{parent.replace('t1_', '')}\" );"
        test = execute_query(connection, ins)
        if test:
            return test

        if (len(replies) > 0):
            test = explore_and_insert_comments(replies, postID)
            if test:
                return test
            
    return False


In [10]:
query = "SELECT * FROM post;"
ALL_Post_list = execute_query_get_result(connection, query)
print("Number of post saved : ", len(ALL_Post_list))

Number of post saved :  373365


In [28]:
query = "SELECT origin_post FROM comments;"
ALL_comments_list = execute_query_get_result(connection, query)
print("Number of comments saved : ", len(ALL_comments_list))

Number of comments saved :  1797934


In [32]:
id_seen = [x[0] for x in ALL_comments_list]
id_seen = list(dict.fromkeys(id_seen))

In [34]:
id = [x[0] for x in ALL_Post_list]
id_formated = [i if i.startswith('t3_') else f't3_{i}' for i in id]

post_gen = reddit.info(fullnames=id_formated)

In [35]:
n = 0
l = 0
try:
    while True:
        i = 0
        for sub in post_gen:
            i += 1
            n += 1
            if id[i] not in id_seen:
                sub.comments.replace_more(limit=None)
                explore_and_insert_comments(sub.comments, id[i])
            if i > 100:
                break

        if i <= 100:
            break
        if n//100 > l:
            l = n//100
            print(n)
except:
    pass

print("Stopped at n = ", n)

101
202
303
404
505
606
707
808
909
1010
1111
1212
1313
1414
1515
1616
1717
1818
1919
2020
2121
2222
2323
2424
2525
2626
2727
2828
2929
3030
3131
3232
3333
3434
3535
3636
3737
3838
3939
4040
4141
4242
4343
4444
4545
4646
4747
4848
4949
5050
5151
5252
5353
5454
5555
5656
5757
5858
5959
6060
6161
6262
6363
6464
6565
6666
6767
6868
6969
7070
7171
7272
7373
7474
7575
7676
7777
7878
7979
8080
8181
8282
8383
8484
8585
8686
8787
8888
8989
9090
9191
9292
9393
9494
9595
9696
9797
9898
9999
10100
10201
10302
10403
10504
10605
10706
10807
10908
11009
11110
11211
11312
11413
11514
11615
11716
11817
11918
12019
12120
12221
12322
12423
12524
12625
12726
12827
12928
13029
13130
13231
13332
13433
13534
13635
13736
13837
13938
14039
14140
14241
14342
14443
14544
14645
14746
14847
14948
15049
15150
15251
15352
15453
15554
15655
15756
15857
15958
16059
16160
16261
16362
16463
16564
16665
16766
16867
16968
17069
17170
17271
17372
17473
17574
17675
17776
17877
17978
18079
18180
18281
18382
18483
18584
1868

### 8: Closing DB connection

In [31]:
connection.close()